In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd 
import numpy as np 
import time
import datetime
import subprocess
import re
import os 
import requests

In [2]:
keywords = ["wikipedia"]
journals = ['information systems research', 'mis quarterly', 'journal of management information systems',
 'journal of the association for information systems', 'management science', 'operational research']  # source: "MISQ"
# journals = ['information systems research']
# authors = ["Kuznets"]  # author:Kuznets
fpath = "/Users/Nico/test/test_googlecrawer"

In [3]:
alias = {'information systems research':"ISR", 'mis quarterly':'MISQ', 'journal of management information systems':"JMIS",
         'journal of the association for information systems':"JAIS", 'management science':'MS', 'operational research':"OR"}

In [4]:
# options = webdriver.ChromeOptions()
# # options.add_argument('headless')

# options.binary_location = '/usr/local/bin/chromedriver'
# options.add_argument('headless')
# options.add_argument("--no-sandbox");
# options.add_argument("--disable-dev-shm-usage")

In [5]:
def getbibTeX(article, driver):
    bib = article.find_element_by_css_selector("div[class=gs_fl]").find_element_by_css_selector("a[class=gs_or_cit\ gs_nph]")
    bib.click()
    time.sleep(2)  ## sleep wait for the ajax to load 
    driver.find_element_by_css_selector("div[id=gs_citi]").find_element_by_css_selector("a[class=gs_citi]").click()
    bib_text = driver.find_element_by_tag_name("body").text.replace("\n", "")
    driver.back()
    driver.find_element_by_css_selector("span[class=gs_ico]").click()
    return bib_text

In [6]:
getbibTeX(article[2], driver)

NameError: name 'article' is not defined

In [ ]:
def getInfo(article, driver):
    default = {"title": "NA", "author": "NA", "journal": "NA", "year": "NA"}
    
    # 提取标题
    try:
        default['title'] = article.find_element(By.CLASS_NAME, "gs_rt").text.lower()
    except:
        pass

    # 提取作者、期刊、年份信息
    try:
        infobox = article.find_element(By.CLASS_NAME, "gs_a").text
        default['author'], default['journal'], default['year'] = parse(infobox)
    except:
        pass
    
    return default

In [ ]:
getInfo(article[0], driver)

{'title': 'economic growth and income inequality',
 'author': 's kuznets',
 'journal': 'the american economic review',
 'year': '1955'}

In [ ]:
def getPdf(article, driver):
    try:
        tmp = article.find_element_by_css_selector("div[class=gs_or_ggsm")
        pdf_link = tmp.find_element_by_tag_name("a").get_attribute("href")
    except:
        pdf_link = "NA"
    return pdf_link

In [7]:
getPdf(article[1], driver)

NameError: name 'getPdf' is not defined

In [8]:
def downloadPdf(output, link):
    response = requests.get(link)
    with open(output, 'wb') as f:
        f.write(response.content)

In [9]:
def parse(infobox):
    infobox = infobox.lower().split("-")
    infobox = [c.strip() for c in infobox]
    author = infobox[0].split(",")[0]
    journal = infobox[1].split(",")[0]
    year = infobox[1].split(",")[1].strip()
    return author, journal, year

In [ ]:
# # for test
# search_keyword = '''online communities'''
# driver = webdriver.Chrome()
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# driver.get('https://scholar.google.com/')
# input_element = driver.find_element(By.NAME ,"q")
# input_element.clear()
# input_element.send_keys(search_keyword)
# input_element.submit()
# elements = driver.find_elements_by_css_selector("div[class=gs_r\ gs_or\ gs_scl]")

In [25]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

search_keyword = "online communities"
driver = webdriver.Chrome()
driver.get('https://scholar.google.com/')

# 输入搜索内容
input_element = driver.find_element(By.NAME, "q")
input_element.clear()
input_element.send_keys(search_keyword)
input_element.submit()

# 等待结果加载出来
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.gs_r.gs_or.gs_scl"))
)

print(f"抓到 {len(elements)} 个结果")
for e in elements:
    print(e.text)


抓到 10 个结果
[PDF] academia.edu
Online communities
R Plant - Technology in society, 2004 - Elsevier
… phenomena, that of the online community. The potential for near … facilitated the development
of online communities which work to … the concept and practice of online communities: first, by …
保存 引用 被引用次数：369 相关文章 所有 10 个版本
[PDF] academia.edu
The anthropology of online communities
SM Wilson, LC Peterson - Annual review of anthropology, 2002 - annualreviews.org
… to these groups, we mustraise several questions: How have scholars approached online
communities andonline communication in general? Is the concept of community itselfmisleading…
保存 引用 被引用次数：1182 相关文章 所有 12 个版本
The real value of online communities
A Armstrong, J Hagel - Knowledge and communities, 2009 - api.taylorfrancis.com
… a community are those businesses that have a substantial economic incentive to exploit
synergies between an online community … of the leading online communities that target children? …
保存 引用 被引用次数：2294 相关文章 所

In [26]:
elements[0].find_elements(By.CLASS_NAME,"gs_fl")

[<selenium.webdriver.remote.webelement.WebElement (session="cbd263a48d4f6ec05ca08f25a84803c6", element="f.1D52A1391B735E567BEBF4BE72C1814F.d.1033CACA8A12CBE1DC69D2D4FDB55354.e.73")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cbd263a48d4f6ec05ca08f25a84803c6", element="f.1D52A1391B735E567BEBF4BE72C1814F.d.1033CACA8A12CBE1DC69D2D4FDB55354.e.74")>]

In [29]:
# 拿到第 6 条搜索结果（索引 5）
tmp = elements[5].find_elements(By.CSS_SELECTOR, ".gs_fl")

# 遍历底部的链接区域，寻找 “被引用次数” 的链接
cite_links = []
for c in tmp:
    try:
        cite = c.find_element(By.CSS_SELECTOR, "a[href^='/scholar?cites']")
        cite_links.append(cite.text)
    except:
        pass

print(cite_links)


['被引用次数：465']


In [59]:
class Article:
    def __init__(self, keywords, target_journal, folder):
        self.keywords = keywords
        self.target_journal = target_journal
        self.output_folder = folder
        self.create_folder()
        self.total_article = {}
    
    def create_folder(self):
        self.output_fpath = "/".join([self.output_folder, self.keywords, self.target_journal])
        if not os.path.exists(self.output_fpath):
            os.makedirs(self.output_fpath)
            print('created folder {0}'.format(self.output_fpath))
    
    def getInfo(self, article, driver):
        default = {"title": "NA", "author": "NA", "journal": "NA", "year":"NA", "log": "NA"}
        
        # 提取标题
        try:
            default['title'] = article.find_element(By.CLASS_NAME, "gs_rt").text.lower()
            # 去掉特殊字符，只保留字母、数字和空格
            default['title'] = re.sub("[^a-z0-9 ]", "", default['title'])
        except:
            pass

        # 提取作者、期刊、年份信息
        try:
            infobox = article.find_element(By.CLASS_NAME, "gs_a").text
            default['author'], default['journal'], default['year'] = parse(infobox)
        except:
            pass

        return default
    
    def getPdf(self, article, driver):
        tmp = article.find_element_by_css_selector("div[class=gs_or_ggsm")
        pdf_link = tmp.find_element_by_tag_name("a").get_attribute("href")
        return pdf_link
    
    def getFileName(self, alias=alias):
        by = ["author", "year", "title", "journal"]
        if alias:
            if self.info['journal'] in alias.keys():
                self.info['journal-short'] = alias[self.info['journal']]
                by = ["author", "year", "title", "journal-short"]
        if len(self.info['title'].split(" ")) > 10:
            self.info['title-short'] = " ".join(self.info['title'].split(" ")[:10])
            if "journal-short" in self.info.keys():
                by = ["author", "year", "title-short", "journal-short"]
            else:
                by = ["author", "year", "title-short", "journal"]
        filename = "-".join([self.info[c] for c in by]) + ".pdf"
        return filename
    
    def fit(self, article, driver, num):
        # try:
        self.info = self.getInfo(article, driver)
        # except:
        #     print("article info parse error!")
        #     self.info = None
        if self.info:
            try:
                self.pdf = self.getPdf(article, driver)
            except:
                self.info['log'] = "pdf missing"
                self.pdf = None
            if self.pdf:
                self.filename = self.getFileName()
                output = self.output_fpath + "/" + self.filename
                try:
                    downloadPdf(output, self.pdf)
                except:
                    self.log = self.info['log'] + "||| pdf download error"
        self.total_article[num] = self.info
#         if hasattr(self, 'log'):
#             now = datetime.datetime.now()
#             logfile_path = self.output_fpath + "/" + "log_{0}.txt".format(now.strftime("%m-%d-%Y"))
#             if not os.path.exists(logfile_path):
#                 subprocess.call("touch {0}".format(logfile_path), shell=True)
#             with open(logfile_path, "a") as f:
#                 f.writelines(self.log)

In [74]:
def run(keywords, journals, recursive = 6):
    driver = webdriver.Chrome()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.get('https://scholar.google.com/')
    for i in keywords:
        for j in journals:
            cnt = 1
            articles = Article(i, j, fpath)
            search_keyword = " ".join([i.lower(), '''source:"{}"'''.format(j.lower())])
            print("current search key: {0}".format(search_keyword))
            input_element = driver.find_element(By.NAME,"q")
            input_element.clear()
            input_element.send_keys(search_keyword)
            input_element.submit()
            time.sleep(2)
            for n in range(recursive):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                elements = driver.find_elements(By.CSS_SELECTOR,"div[class=gs_r\ gs_or\ gs_scl]")
                for e in elements:
                    # try:
                    articles.fit(e, driver, cnt)
                    # except:
                    #     print("page {} number {} parse error!".format(n, cnt))
                    cnt += 1
                # try:
                    driver.find_element(By.CSS_SELECTOR, "span.gs_ico.gs_ico_nav_next")
                    time.sleep(5)
                # except:
                #     pass
            log = pd.DataFrame(articles.total_article).T
            now = datetime.datetime.now()
            log.to_csv(articles.output_fpath+"/"+"logfile_{}.txt".format(now.strftime("%m-%d-%Y")), sep="\t")
    driver.quit()

In [75]:
# article = driver.find_element_by_css_selector("div[class^=gs_ri")

In [76]:
journals = ['journal of management information systems',
 'journal of the association for information systems', 'management science', 'operational research']

In [78]:
run(keywords, journals, recursive=6)

current search key: wikipedia source:"journal of management information systems"


InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff67f1f3d85+79397]
	GetHandleVerifier [0x0x7ff67f1f3de0+79488]
	(No symbol) [0x0x7ff67ef9bf2c]
	(No symbol) [0x0x7ff67efe32df]
	(No symbol) [0x0x7ff67f01b062]
	(No symbol) [0x0x7ff67f015b93]
	(No symbol) [0x0x7ff67f014c59]
	(No symbol) [0x0x7ff67ef66fd5]
	GetHandleVerifier [0x0x7ff67f4be1ed+3005069]
	GetHandleVerifier [0x0x7ff67f4b831d+2980797]
	GetHandleVerifier [0x0x7ff67f4d7e0d+3110573]
	GetHandleVerifier [0x0x7ff67f20d6de+184190]
	GetHandleVerifier [0x0x7ff67f21516f+215567]
	(No symbol) [0x0x7ff67ef65fd1]
	GetHandleVerifier [0x0x7ff67f5e1d48+4199912]
	BaseThreadInitThunk [0x0x7ffff0a07374+20]
	RtlUserThreadStart [0x0x7ffff19bcc91+33]
